In [36]:
import geopandas as gpd
import pandas as pd
from geopy.distance import great_circle
import requests
import time

In [50]:
rent_df = pd.read_csv('../../data/landing/cleaned_rent_data.csv')
hospital_df = pd.read_csv('../../data/curated/hospitals.csv')
rent_df['nearestHospital'] = None

In [51]:
rent_df

,price,bathrooms,bedrooms,parking,type,street,suburb,postcode,latitude,longitude,school,furnished,SA2_code,nearestHospital
0,750.0,2,2,0,Apartment / Unit / Flat,403/639 Lonsdale Street,MELBOURNE,3000.0,-37.815010,144.95341,Docklands Primary School1.1 km awayPrep-6CoEdG...,True,206041505.0,None
1,625.0,1,1,0,Apartment / Unit / Flat,3409/138 Spencer Street,MELBOURNE,3000.0,-37.817470,144.95366,Docklands Primary School1.2 km awayPrep-6CoEdG...,True,206041505.0,None
2,620.0,1,1,1,Apartment / Unit / Flat,103/300 Swanston Street,MELBOURNE,3000.0,-37.810608,144.96458,Carlton Gardens Primary School1 km awayPrep-6C...,True,206041504.0,None
3,550.0,1,1,0,Apartment / Unit / Flat,801/115 Swanston Street,MELBOURNE,3000.0,-37.814995,144.96603,Carlton Gardens Primary School1.5 km awayPrep-...,True,206041503.0,None
4,525.0,1,1,0,Apartment / Unit / Flat,808/422 Collins Street,MELBOURNE,3000.0,-37.817097,144.96014,Docklands Primary School1.7 km awayPrep-6CoEdG...,False,206041505.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11649,420.0,1,3,1,House,48 Hunter Street,WONTHAGGI,3995.0,-38.613125,145.58660,Wonthaggi Primary School0.6 km awayPrep-6CoEdG...,False,205031093.0,None
11650,550.0,2,3,2,House,64 Halford Street,INVERLOCH,3996.0,-38.634730,145.71307,Inverloch Primary School1.4 km awayPrep-6CoEdG...,False,205031093.0,None
11651,450.0,1,3,0,House,35 Anderson Avenue,INVERLOCH,3996.0,-38.629770,145.73927,Inverloch Primary School0.9 km awayPrep-6CoEdG...,False,205031093.0,None
11652,435.0,1,3,0,House,3 Kennards Court,INVERLOCH,3996.0,-38.626730,145.72865,Inverloch Primary School0.5 km awayPrep-6CoEdG...,True,205031093.0,None


In [52]:
hospital_df

,Hospital name,Phone number,Street address,Suburb,Postcode,State,Local Hospital Network (LHN),Primary Health Network area (PHN),Website,Description,Sector,Beds,Latitude,Longitude,Hospital_ID
0,Albert Road Clinic,03 9256 8311,31 Albert Road,Melbourne,3004,Vic,NaN,NaN,www.albertroadclinic.com.au/,NaN,Private,50-99,-37.834246,144.972151,8
1,Albury Wodonga Health [Wodonga Campus],02 6051 7111,69 Vermont Street,Wodonga,3690,Vic,Albury Wodonga Health,Murray,NaN,Medium regional hospital with an emergency dep...,Public,100-199,-36.131791,146.879417,10
2,Alexandra District Hospital,03 5772 0900,12 Cooper Street,Alexandra,3714,Vic,Alexandra District Hospital,Murray,NaN,Small hospital,Public,<50,-37.195065,145.716143,12
3,Alpine Health [Bright],03 5755 0100,36-42 Cobden Street,Bright,3741,Vic,Alpine Health (Vic),Murray,www.alpinehealth.org.au,Small hospital,Public,<50,-36.732701,146.965924,16
4,Alpine Health [Mount Beauty],03 5754 3500,10 Hollonds Street,Mt Beauty,3699,Vic,Alpine Health (Vic),Murray,www.alpinehealth.org.au,Regional hospital,Public,<50,-36.743354,147.169359,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,Women's at Sandringham,03 9076 1000,193 Bluff Road,Sandringham,3191,Vic,Royal Women's Hospital (Melbourne),South Eastern Melbourne,www.thewomens.org.au/health-professionals/mate...,Specialist women's hospital,Public,<50,-37.961049,145.018157,995
234,Yarra Ranges Health,1300 342 255,25 Market Street,Lilydale,3140,Vic,Eastern Health (Vic),Eastern Melbourne,https://www.easternhealth.org.au/locations/yar...,Day procedure hospital,Public,<50,-37.756319,145.351204,1005
235,Yarram & District Health Service,03 5182 0222,85-91 Commercial Road,Yarram,3971,Vic,Yarram and District Health Service,Gippsland,www.ydhs.com.au/,Regional hospital,Public,<50,-38.556975,146.678316,1006
236,Yarrawonga District Health Service,03 5743 8111,33 Piper Street,Yarrawonga,3730,Vic,Yarrawonga District Health Service,Murray,http://www.yh.org.au/,Small hospital,Public,<50,-36.011547,146.006584,1007


In [57]:
def the_nearest_location(rentDf, locationDf, locationName):
    """ 
    Function to find the nearest location (e.g., hospital, school)
    Parameters: 
        rentDf: DataFrame containing properties with latitude and longitude.
        locationDf: DataFrame containing locations (e.g., hospitals) with latitude and longitude.
        locationName: The column name to store the nearest location information.
    
    Return: 
        Updated DataFrame with nearest location and distance added in the specified column.
    """
    # Loop through each property in rentDf
    for index, row in rentDf.iterrows():
        point1 = (row['latitude'], row['longitude'])  # Property's coordinates
        min_distance = float('inf')  # Set initial minimum distance to infinity
        nearest_location = None  # Store nearest location's coordinates

        # Loop through each location in locationDf to calculate the distance
        for index2, row2 in locationDf.iterrows():
            point2 = (row2['Latitude'], row2['Longitude'])  # Location's coordinates
            distance = great_circle(point1, point2).kilometers  # Calculate distance using great_circle
            
            # Update minimum distance if a closer location is found
            if distance < min_distance:
                min_distance = distance
                nearest_location = point2

        # Create a dictionary to store nearest location information
        info_dict = {
            'dist': min_distance,
            'nearest_coord': nearest_location
        }

        # Update rentDf with the nearest location information
        rentDf.at[index, locationName] = info_dict

    return rentDf


In [58]:
rent_df = the_nearest_location(rent_df, hospital_df, 'nearestHospital')

In [59]:
rent_df['hospital_distance'] = None
rent_df['to_hospital_time'] = None

In [60]:
rent_df

,price,bathrooms,bedrooms,parking,type,street,suburb,postcode,latitude,longitude,school,furnished,SA2_code,nearestHospital,hospital_distance,to_hospital_time
0,750.0,2,2,0,Apartment / Unit / Flat,403/639 Lonsdale Street,MELBOURNE,3000.0,-37.815010,144.95341,Docklands Primary School1.1 km awayPrep-6CoEdG...,True,206041505.0,"{'dist': 0.34167395403372697, 'nearest_coord':...",None,None
1,625.0,1,1,0,Apartment / Unit / Flat,3409/138 Spencer Street,MELBOURNE,3000.0,-37.817470,144.95366,Docklands Primary School1.2 km awayPrep-6CoEdG...,True,206041505.0,"{'dist': 0.5822133115824419, 'nearest_coord': ...",None,None
2,620.0,1,1,1,Apartment / Unit / Flat,103/300 Swanston Street,MELBOURNE,3000.0,-37.810608,144.96458,Carlton Gardens Primary School1 km awayPrep-6C...,True,206041504.0,"{'dist': 0.7376496705051538, 'nearest_coord': ...",None,None
3,550.0,1,1,0,Apartment / Unit / Flat,801/115 Swanston Street,MELBOURNE,3000.0,-37.814995,144.96603,Carlton Gardens Primary School1.5 km awayPrep-...,True,206041503.0,"{'dist': 0.47429311644965033, 'nearest_coord':...",None,None
4,525.0,1,1,0,Apartment / Unit / Flat,808/422 Collins Street,MELBOURNE,3000.0,-37.817097,144.96014,Docklands Primary School1.7 km awayPrep-6CoEdG...,False,206041505.0,"{'dist': 0.9547837766075489, 'nearest_coord': ...",None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11649,420.0,1,3,1,House,48 Hunter Street,WONTHAGGI,3995.0,-38.613125,145.58660,Wonthaggi Primary School0.6 km awayPrep-6CoEdG...,False,205031093.0,"{'dist': 0.6940849282484552, 'nearest_coord': ...",None,None
11650,550.0,2,3,2,House,64 Halford Street,INVERLOCH,3996.0,-38.634730,145.71307,Inverloch Primary School1.4 km awayPrep-6CoEdG...,False,205031093.0,"{'dist': 11.839661644936594, 'nearest_coord': ...",None,None
11651,450.0,1,3,0,House,35 Anderson Avenue,INVERLOCH,3996.0,-38.629770,145.73927,Inverloch Primary School0.9 km awayPrep-6CoEdG...,False,205031093.0,"{'dist': 13.955512102405784, 'nearest_coord': ...",None,None
11652,435.0,1,3,0,House,3 Kennards Court,INVERLOCH,3996.0,-38.626730,145.72865,Inverloch Primary School0.5 km awayPrep-6CoEdG...,True,205031093.0,"{'dist': 12.9907802359375, 'nearest_coord': (-...",None,None


In [61]:
hospital_df

,Hospital name,Phone number,Street address,Suburb,Postcode,State,Local Hospital Network (LHN),Primary Health Network area (PHN),Website,Description,Sector,Beds,Latitude,Longitude,Hospital_ID
0,Albert Road Clinic,03 9256 8311,31 Albert Road,Melbourne,3004,Vic,NaN,NaN,www.albertroadclinic.com.au/,NaN,Private,50-99,-37.834246,144.972151,8
1,Albury Wodonga Health [Wodonga Campus],02 6051 7111,69 Vermont Street,Wodonga,3690,Vic,Albury Wodonga Health,Murray,NaN,Medium regional hospital with an emergency dep...,Public,100-199,-36.131791,146.879417,10
2,Alexandra District Hospital,03 5772 0900,12 Cooper Street,Alexandra,3714,Vic,Alexandra District Hospital,Murray,NaN,Small hospital,Public,<50,-37.195065,145.716143,12
3,Alpine Health [Bright],03 5755 0100,36-42 Cobden Street,Bright,3741,Vic,Alpine Health (Vic),Murray,www.alpinehealth.org.au,Small hospital,Public,<50,-36.732701,146.965924,16
4,Alpine Health [Mount Beauty],03 5754 3500,10 Hollonds Street,Mt Beauty,3699,Vic,Alpine Health (Vic),Murray,www.alpinehealth.org.au,Regional hospital,Public,<50,-36.743354,147.169359,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,Women's at Sandringham,03 9076 1000,193 Bluff Road,Sandringham,3191,Vic,Royal Women's Hospital (Melbourne),South Eastern Melbourne,www.thewomens.org.au/health-professionals/mate...,Specialist women's hospital,Public,<50,-37.961049,145.018157,995
234,Yarra Ranges Health,1300 342 255,25 Market Street,Lilydale,3140,Vic,Eastern Health (Vic),Eastern Melbourne,https://www.easternhealth.org.au/locations/yar...,Day procedure hospital,Public,<50,-37.756319,145.351204,1005
235,Yarram & District Health Service,03 5182 0222,85-91 Commercial Road,Yarram,3971,Vic,Yarram and District Health Service,Gippsland,www.ydhs.com.au/,Regional hospital,Public,<50,-38.556975,146.678316,1006
236,Yarrawonga District Health Service,03 5743 8111,33 Piper Street,Yarrawonga,3730,Vic,Yarrawonga District Health Service,Murray,http://www.yh.org.au/,Small hospital,Public,<50,-36.011547,146.006584,1007


# Calculate distances using the OpenRoute Service
### Due to the API request limitations (2000 requests per day, 40 per minute), we decided to divide the properties into 5 subsets. Each member will be responsible for calculating the distances individually using their own account.

### Please note that in the code below, we are working with the entire dataset, and once the loop reaches 2000 requests, an API request error will occur.

In [ ]:
import time
import requests
import pandas as pd
import os

# Your API key (hidden for security reasons)
your_api = os.getenv('ORS_API_KEY')  # Use environment variable to store the API key

# API request limit parameters
max_per_Min = 40
time_sleep = 60 / max_per_Min

# Define the batch range, for example [1, 2000]
start = 10001
end = 11655

# Ensure the save file path exists
output_dir = '../../data/External_data/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)  # Create directory if it does not exist

# Iterate over each entry in the rental dataset for the specified index range
for index, row in rent_df.iloc[start-1:end].iterrows():  # Note iloc starts at 0, so start-1
    if pd.isna(row['hospital_distance']):  # Only process records where hospital_distance is empty
        longi = row['longitude']
        latit = row['latitude']

        # Get the coordinates of the nearest hospital
        nearest_hospital_info = row['nearestHospital']
        if nearest_hospital_info and 'nearest_coord' in nearest_hospital_info:
            hospital_longi = nearest_hospital_info['nearest_coord'][1]  # Longitude
            hospital_latit = nearest_hospital_info['nearest_coord'][0]  # Latitude

            # Construct the API request URL
            api_url = f'https://api.openrouteservice.org/v2/directions/driving-car?api_key={your_api}&start={longi},{latit}&end={hospital_longi},{hospital_latit}'

            try:
                # Send the API request
                response = requests.get(api_url)

                # Check the request status
                if response.status_code == 200:
                    info = response.json()

                    # Parse the API response
                    features = info.get('features', [])
                    if features:
                        summary = features[0]['properties']['summary']  # Get driving route information
                        distance = summary['distance']  # Distance, in meters
                        duration_time = summary['duration']  # Driving time, in seconds

                        # Update rent_df with hospital_distance and to_hospital_time
                        rent_df.at[index, 'hospital_distance'] = distance
                        rent_df.at[index, 'to_hospital_time'] = duration_time
                else:
                    print(f"Request failed with status code: {response.status_code}")

            except requests.exceptions.RequestException as e:
                print(f"API request failed: {e}")

            # Limit the API request rate
            time.sleep(time_sleep)

    else:
        continue

# Save the current batch of data to the specified directory
output_file = f'{output_dir}rent_data_{start}_to_{end}.csv'
rent_df.iloc[start-1:end].to_csv(output_file, index=False)

print(f'Data saved to {output_file}')